In [1]:
import os
import torch

from PIL import Image
import open_clip

In [2]:
name = "ViT-B-16-plus-240"
pretrained = "laion400m_e32"
model, train_transform, eval_transform = open_clip.create_model_and_transforms(name, pretrained=pretrained)

In [3]:
from pathlib import Path

ouptut_dir = "openvino_int8"
if not os.path.exists(ouptut_dir):
    os.makedirs(ouptut_dir)
with open(Path(ouptut_dir) / "model_index.txt", 'w') as fd:
    fd.write(f"{name}, {pretrained}\n")

In [4]:
tokenizer = open_clip.get_tokenizer('ViT-B-16-plus-240')

image = eval_transform(Image.open("../docs/CLIP.png")).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a cat"])

### Prepare data for optimization

In [5]:
import os
import random
from io import BytesIO
import requests
import numpy as np

def get_pil_from_url(url):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    return image.convert("RGB")

BACKUP_PAIR = (
    get_pil_from_url(
        "https://thumbs.dreamstime.com/t/altai-mountains-mountain-lake-russia-siberia-chuya-ridge-49130812.jpg"
    ),
    "Altai mountains Stock Photography",
)
AVAILABLE_EXAMPLES = []

def check_text_data(data):
    if isinstance(data, str):
        return True
    if isinstance(data, list):
        return all(isinstance(x, str) for x in data)
    return False    

def laion2B_preprocess_train(examples, train_transforms, tokenize_captions, image_column="url", text_column="caption"):
    url = examples[image_column]
    try:
        image = get_pil_from_url(url)
        if not check_text_data(examples[text_column]):
            raise ValueError("Text data is not valid")
        AVAILABLE_EXAMPLES.append((url, examples[text_column]))
    except Exception:
        print(f"Can't load image from url: {url}, using cache with size: {len(AVAILABLE_EXAMPLES)}")
        if len(AVAILABLE_EXAMPLES) > 0:
            backup_id = random.randint(0, len(AVAILABLE_EXAMPLES) - 1)
            backup_example = AVAILABLE_EXAMPLES[backup_id]
            try:
                image = get_pil_from_url(backup_example[0])
                examples[text_column] = backup_example[1]
            except Exception:
                print(f"Can't load image from cached url: {backup_example[0]}, using backup")
                image = BACKUP_PAIR[0].copy()
                examples[text_column] = BACKUP_PAIR[1]
        else:
            print(f"Can't load image from url: {url}, using backup")
            image = BACKUP_PAIR[0].copy()
            examples[text_column] = BACKUP_PAIR[1]

    examples["pixel_values"] = train_transforms(image)
    examples["text"] = tokenize_captions(examples)
    return examples

def tokenize_captions(examples, is_train=True):
    caption_column = "caption"
    captions = []
    caption = examples[caption_column]
    if isinstance(caption, str):
        captions.append(caption)
    elif isinstance(caption, (list, np.ndarray)):
        # take a random caption if there are multiple
        captions.append(random.choice(caption) if is_train else caption[0])
    else:
        raise ValueError(f"Caption column `{caption_column}` should contain either strings or lists of strings.")
    #inputs = tokenizer(captions[0], max_length=tokenizer.model_max_length, padding="do_not_pad", truncation=True)
    #input_ids = inputs.input_ids
    input_ids = tokenizer(captions[0])[0]
    return input_ids

In [6]:
from datasets import load_dataset

max_train_samples = 10000
dataset = load_dataset("laion/laion400m", streaming=True)
train_dataset = dataset["train"].shuffle(seed=42, buffer_size=max_train_samples)

In [7]:
cast_dtype = model.transformer.get_cast_dtype()

def preprocess_text(text):
    with torch.no_grad():
        x = model.token_embedding(text).to(cast_dtype)  # [batch_size, n_ctx, d_model]
        x = x + model.positional_embedding.to(cast_dtype)
    return x

def collate_fn_image(examples):
    examples = [laion2B_preprocess_train(example, train_transform, tokenize_captions) for example in examples]
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
    
    input_ids = torch.stack([preprocess_text(example["text"]) for example in examples]).permute(1, 0, 2) # NLD -> LND
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_masks": model.attn_mask,
    }

In [8]:
import itertools
from tqdm.notebook import tqdm

def prepare_nncf_init_data(dataloader, init_steps):
    nncf_init_data = []

    print(f"Fetching {init_steps} for the initialization...")
    for _, batch in tqdm(zip(range(init_steps), itertools.islice(dataloader, 0, init_steps))):
        with torch.no_grad():
            # Convert images to latent space
            
            nncf_init_data.append(
                (
                    batch["pixel_values"].to("cpu"),
                    batch["input_ids"].to("cpu"),
                    batch["attention_masks"].to("cpu")
                )
            )
    return nncf_init_data

In [9]:
train_batch_size = 1
dataloader_num_workers = 4
train_dataloader = torch.utils.data.DataLoader(
        train_dataset, collate_fn=collate_fn_image, batch_size=train_batch_size, num_workers=dataloader_num_workers
    )

In [10]:
opt_init_steps = 300
init_data = prepare_nncf_init_data(train_dataloader, opt_init_steps)

Fetching 300 for the initialization...


0it [00:00, ?it/s]

Can't load image from url: https://s3.amazonaws.com/whataspace/space_pictures/pictures/000/033/180/fullwidth/_DSC0146.jpg?1591610094, using cache with size: 0
Can't load image from url: https://s3.amazonaws.com/whataspace/space_pictures/pictures/000/033/180/fullwidth/_DSC0146.jpg?1591610094, using backup
Can't load image from url: http://cdn3.static-homes.com/cgi-bin/readimage/9eb0ae4fa92b5dfd09b03c9e3dc997c5_1_resizeto_193x143x1, using cache with size: 3
Can't load image from url: https://i0.wp.com/des.gearbest.com/uploads/pdm-desc-pic/Electronic/image/2016/11/25/1480056293210706.jpg?w=960, using cache with size: 2
Can't load image from url: https://sslh.ulximg.com/image/740x493/cover/1533483761_ac8b9fbc89ca57a7269ec0c7f5947094.jpg/2143af354a3c53d14969369d2c6cbc04/1533483761_cc5f2e1e234c809fa4408488e0b19e4d.jpg, using cache with size: 7
Can't load image from url: https://img.shellporn.com/spcs/thumbs/155/299_hotel_wang_.jpg, using cache with size: 5
Can't load image from url: http://r

/home/alex/virt_envs/tomeov/lib/python3.8/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Can't load image from url: http://i0.wp.com/www.ewindandsolar.com/i/2015/06/hy-380-led-puck-lights-for-cozy-living-room-design-led-puck-light-kit-under-cabinet-led-puck-lights-12v-led-puck-lights-led-puck-lights-120v-battery-powered-led-puck-lights-battery-led.jpg?w=200&strip=all, using cache with size: 18
Can't load image from url: https://cdn1-www.momtastic.com/assets/uploads/2014/01/old-DIYs-e1389240447729.jpg, using cache with size: 22
Can't load image from url: https://www.belleviemedical.com/wp-content/uploads/2016/03/Belle-Vie-Home-Face.png, using cache with size: 18
Can't load image from url: http://s.en.fishki.net/upload/en/201211/23/2706/9840483e0be437697bcdb6501908d666.jpg, using cache with size: 19
Can't load image from url: https://d2d00szk9na1qq.cloudfront.net/Product/0d387c2a-4687-4900-8861-17a0df831812/Images/Medium_0393423.jpg, using cache with size: 23
Can't load image from url: https://d39eittn6ocpe0.cloudfront.net/media/catalog/product/cache/67df2192fc19fe888eca63b2

In [11]:
class InitDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        super().__init__()
        self.init_data = data

    def __len__(self):
        return len(self.init_data)

    def __getitem__(self, index):
        return self.init_data[index]

Can't load image from url: http://ep.yimg.com/ay/yhst-83532116742892/cherry-blossom-birdies-pink-yellow-framed-art-print-8.jpg, using cache with size: 62


### Quantize Image Encoder

In [12]:
import nncf

def quantize_image_encoder(model, data_loader):
    quantization_dataset = nncf.Dataset(data_loader, lambda x: x[0])

    quantized_model = nncf.quantize(
                            model,
                            quantization_dataset,
                            model_type=nncf.ModelType.TRANSFORMER,
                            preset=nncf.QuantizationPreset.MIXED,
                            
                            )
    return quantized_model

/home/alex/virt_envs/tomeov/lib/python3.8/site-packages/openvino/offline_transformations/__init__.py:10: FutureWarning: The module is private and following namespace `offline_transformations` will be removed in the future, use `openvino.runtime.passes` instead!
  warnings.warn(


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


In [13]:
import openvino.runtime as ov
from pathlib import Path

ov_model_path = Path("openvino_fp32/image_encoder.xml")

core = ov.Core()
image_encoder = core.read_model(ov_model_path)

q_image_encoder = quantize_image_encoder(image_encoder, InitDataset(init_data))

INFO:nncf:212 ignored nodes was found by types in the NNCFGraph
INFO:nncf:Not adding activation input quantizer for operation: 6 /Add_1
INFO:nncf:Not adding activation input quantizer for operation: 9 /ln_pre/Div
11 /ln_pre/Mul
13 /ln_pre/Add_1

INFO:nncf:Not adding activation input quantizer for operation: 17 /transformer/resblocks.0/ln_1/Div
20 /transformer/resblocks.0/ln_1/Mul
24 /transformer/resblocks.0/ln_1/Add_1

INFO:nncf:Not adding activation input quantizer for operation: 124 /transformer/resblocks.0/attn/Div_2
INFO:nncf:Not adding activation input quantizer for operation: 16 /transformer/resblocks.0/Add
INFO:nncf:Not adding activation input quantizer for operation: 19 /transformer/resblocks.0/ln_2/Div
23 /transformer/resblocks.0/ln_2/Mul
28 /transformer/resblocks.0/ln_2/Add_1

INFO:nncf:Not adding activation input quantizer for operation: 18 /transformer/resblocks.0/Add_1
INFO:nncf:Not adding activation input quantizer for operation: 22 /transformer/resblocks.1/ln_1/Div
27 /t

100%|██████████| 300/300 [01:01<00:00,  4.87it/s]


In [ ]:
ov.serialize(q_image_encoder, ouptut_dir + "/image_encoder.xml")

### Quantize Text Encoder

In [ ]:
def quantize_text_encoder(model, data_loader):
    quantization_dataset = nncf.Dataset(data_loader, lambda x: (x[1], x[2]))

    quantized_model = nncf.quantize(
                            model,
                            quantization_dataset,
                            model_type=nncf.ModelType.TRANSFORMER,
                            preset=nncf.QuantizationPreset.MIXED,
                            
                            )
    return quantized_model

In [ ]:
ov_model_path = Path("openvino_fp32/text_encoder.xml")

core = ov.Core()
text_encoder = core.read_model(ov_model_path)

q_text_encoder = quantize_text_encoder(text_encoder, InitDataset(init_data))

INFO:nncf:204 ignored nodes was found by types in the NNCFGraph
INFO:nncf:Not adding activation input quantizer for operation: 3 /resblocks.0/ln_1/Div
18 /resblocks.0/ln_1/Mul
34 /resblocks.0/ln_1/Add_1

INFO:nncf:Not adding activation input quantizer for operation: 167 /resblocks.0/attn/Div_1
INFO:nncf:Not adding activation input quantizer for operation: 19 /resblocks.0/attn/Add_2
INFO:nncf:Not adding activation input quantizer for operation: 2 /resblocks.0/Add
INFO:nncf:Not adding activation input quantizer for operation: 17 /resblocks.0/ln_2/Div
33 /resblocks.0/ln_2/Mul
50 /resblocks.0/ln_2/Add_1

INFO:nncf:Not adding activation input quantizer for operation: 16 /resblocks.0/Add_1
INFO:nncf:Not adding activation input quantizer for operation: 32 /resblocks.1/ln_1/Div
49 /resblocks.1/ln_1/Mul
67 /resblocks.1/ln_1/Add_1

INFO:nncf:Not adding activation input quantizer for operation: 219 /resblocks.1/attn/Div_1
INFO:nncf:Not adding activation input quantizer for operation: 20 /resblock

100%|██████████| 300/300 [00:20<00:00, 14.96it/s]


In [ ]:
ov.serialize(q_text_encoder, ouptut_dir + "/text_encoder.xml")